In [ ]:
!nvidia-smi

In [ ]:
# !pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [ ]:
# !pip install --upgrade accelerate
# !pip uninstall -y transformers accelerate
# !pip install transformers accelerate

In [ ]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset

from transformers import FlaxAutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [ ]:
#download and unzip the data

!wget = https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
!unzip summarizer-data.zip

In [ ]:
dataset_samsum = load_from_disk('samsum_dataset')
dataset_samsum

In [ ]:
split_lengths = [len(dataset_samsum[split])for split in dataset_samsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue: ")

print(dataset_samsum["test"][1]["dialogue"])

print("\nSummary: ")

print(dataset_samsum["test"][1]["summary"])

In [ ]:
def convert_examples_to_features(example_batch):
  input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

  return {
      'input_ids' : input_encodings['input_ids'],
      'attention_mask': input_encodings['attention_mask'],
      'labels': target_encodings['input_ids']
  }

In [ ]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)

In [ ]:
dataset_samsum_pt["train"]

In [ ]:
#training

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)

In [ ]:
from transformers import TrainingArguments, Trainer


trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, eval_strategy='steps', eval_steps=500, save_steps=1e6,
    logging_steps=10,
    gradient_accumulation_steps=16
)

In [ ]:
trainer = Trainer(model=model_pegasus, args = trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"]
                  )

In [ ]:
trainer.train()

In [ ]:
#Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
  """Split the dataset into smaller batches that we can process simultaneously
  Yield successive batch-sized chunks from list_of_elements."""

  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i : i + batch_size]


def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                                batch_size=16, device='cuda',
                                column_text = "dialogue",
                                column_summary = "summary" ):
  # Extract dialogue and summary columns as lists from the dataset
  dialogues = dataset[column_text]
  summaries = dataset[column_summary]

  dialogue_batches = list(generate_batch_sized_chunks(dialogues, batch_size))
  summary_batches = list(generate_batch_sized_chunks(summaries, batch_size))

  for dialogue_batch, summary_batch in tqdm(
      zip(dialogue_batches, summary_batches), total=len(dialogue_batches)):
      inputs= tokenizer(dialogue_batch, max_length=1024, truncation=True,
                        padding="max_length", return_tensors="pt")
      generated_summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                 attention_mask = inputs["attention_mask"].to(device),
                                 length_penalty = 0.8, num_beams = 8, max_length =128)

      decode_summaries = [tokenizer.decode(s,skip_special_tokens=True,
                                          clean_up_tokenization_spaces=True)
                          for s in generated_summaries]

      metric.add_batch(predictions=decode_summaries, references=summary_batch)

  score = metric.compute()
  return score

In [ ]:
# !pip install evaluate
from evaluate import load

rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load('rouge')

In [ ]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)


rouge_dict = dict((rn, score[rn]) for rn in rouge_names)

pd.DataFrame(rouge_dict, index=[f'pegasus'])

In [ ]:
#Save Model
model_pegasus.save_pretrained("pegasus-samsum-model")

In [ ]:
#save tokenizer
tokenizer.save_pretrained("tokenizer")

In [ ]:
#Load

tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [ ]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

sample_text = dataset_samsum["test"][0]["dialogue"]

reference = dataset_samsum["test"][0]["summary"]

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)


print("Dialogue: ")
print(sample_text)


print("\nReference Summary: ")
print(reference)


print("\nModel Summary: ")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])